# Web Scraping and FastAPI Setup for Country Data from World Bank

In [ ]:
!pip install requests beautifulsoup4 selenium pandas fastapi uvicorn

## Step 1: Scraping the World Bank Website using Selenium

In [ ]:

import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Selenium WebDriver Setup (Make sure to download the correct chromedriver for your Chrome version)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # To run the browser in headless mode
driver = webdriver.Chrome(options=options)

# Open the World Bank's country data page
url = 'https://data.worldbank.org/country'
driver.get(url)
time.sleep(5)  # Let the page load

# Parse the loaded page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract country names and other relevant information
countries_data = []

# Assuming country names are inside certain tags (you might need to inspect the HTML structure)
for country_section in soup.find_all('div', class_='country-data'):
    country_name = country_section.find('h3').get_text(strip=True)
    
    # You would similarly scrape other data points if available
    gdp = country_section.find('span', class_='gdp').get_text(strip=True) if country_section.find('span', class_='gdp') else "N/A"
    population = country_section.find('span', class_='population').get_text(strip=True) if country_section.find('span', class_='population') else "N/A"
    unemployment_rate = country_section.find('span', class_='unemployment').get_text(strip=True) if country_section.find('span', class_='unemployment') else "N/A"
    
    countries_data.append({
        'name': country_name,
        'gdp': gdp,
        'population': population,
        'unemployment_rate': unemployment_rate
    })

# Convert data to a DataFrame or store in a CSV
df = pd.DataFrame(countries_data)
df.to_csv('countries_data.csv', index=False)

# Close the browser
driver.quit()
    

## Step 2: Setting up FastAPI to Serve Scraped Data

In [ ]:

from fastapi import FastAPI, HTTPException
import pandas as pd

# Load the scraped data from CSV
df = pd.read_csv('countries_data.csv')

# Convert DataFrame to a list of dictionaries
countries_data = df.to_dict(orient='records')

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Welcome to the World Bank Country API"}

# Endpoint to get data for a specific country
@app.get("/country/{country_name}")
def get_country_data(country_name: str):
    # Filter for the country in the dataset
    country = next((country for country in countries_data if country["name"].lower() == country_name.lower()), None)
    if country is None:
        raise HTTPException(status_code=404, detail="Country not found")
    return country

# To run the FastAPI server, use Uvicorn in terminal:
# uvicorn your_script_name:app --reload
    